# Stan obiektów w Rubim

<br/>

## dr inż. Aleksander Smywiński-Pohl

## apohllo@agh.edu.pl

## http://apohllo.pl/dydaktyka/programowanie-obiektowe

## konsultacje: środa 17-18

<center><img src="img/ruby-horizontal.svg" style="width: 800px" /></center>

In [6]:
class Speed
  def initialize(value)
    @value = value
  end
  
  def to_s
    "Speed: " + @value.to_s
  end
end

:to_s

W Rubym odpowiednikiem konstruktora jest metoda `initialize`. Odpowiednikiem `toString` z Javy jest `to_s`.
Zmienne instancyjne zaczynają się od @. 

In [7]:
speed1 = Speed.new(10)
speed2 = Speed.new(20)

puts speed1
puts speed2
puts speed1

Speed: 10
Speed: 20
Speed: 10


Utworzenie nowego obiektu w Rubym odbywa się poprzez wywołanie `NazwaKlasy.new` - `new` jest niemalże "zwykłą" metodą. Wywołanie `puts` niejawnie korzysta z metody `to_s`.

In [8]:
class Speed
  def initialize(value, unit)
    @value = value
    @unit = unit
  end
  
  def to_s
    "Speed #{@value} #{@unit}"
  end
  
  def +(other)
    if @unit == other.@unit                   # <--------------!
      Speed.new(@value + other.@value, @unit) # <--------------!
    else
      self # lepiej rzucić wyjątek
    end
  end
end

SyntaxError: unexpected instance variable
    if @unit == other.@unit                   # <-------...
                      ^~~~~
SyntaxError: unexpected instance variable
...Speed.new(@value + other.@value, @unit) # <--------------!
...                         ^~~~~~
SyntaxError: unexpected ')', expecting '='
...w(@value + other.@value, @unit) # <--------------!
...                              ^
SyntaxError: unexpected `end', expecting end-of-input


Kod powoduje błąd składniowy ponieważ w Rubym nie można odwołać się do atrybutów obiektu inaczej niż przez niekwalifikowaną nazwę atrybutu (`@atrybut`). `zmienna.@atrybut` jest konstrukcją niepoprawną składniowo - w ten sposób zagwarantowana jest prywatność atrybutów - stan - obiektów.

<center><img src="img/ruby_variable.png" /></center>

Widzimy tutaj istotną różnicę względem Javy - w tym języku inne obiekty tej samej klasy mają dostęp do prywatnych atrybutów innych obiektów tej samej klasy. W Rubym nie ma takiego dostępu.

In [9]:
class Speed
  attr_reader :unit, :value
    
  def initialize(value, unit)
    @value = value
    @unit = unit
  end
    
  def to_s
    "Speed #{@value} #{@unit}"
  end
  
  def +(other)
    if @unit == other.unit()
      Speed.new(@value + other.value(), @unit)
    else
      self
    end
  end
end

:+

Aby odwołać się do wartości atrybutu w innym obiekcie musimy skorzystać z pośrednictwa metody. 
`attr_reader` definiuje nam gettery - w tym wypadku dla atrybutów `unit` oraz `value`. Należy jednak mieć na uwadze, że wywołanie `other.unit` i `other.value` nie odwołują się bezpośrednio do wartości atrybutów lecz korzystają z pośrednictwa zdefiniowanych metod (które mogą być nadpisane).

W Rubym można redefiniować operatory. `+(other)` definiuje nam operator, który można wywoływać na obiektach klasy `Speed`.

In [10]:
speed1 = Speed.new(10, :KMH)
speed2 = Speed.new(20, :KMH)

speed3 = speed1 + speed2

puts speed3

Speed 30 KMH


<center><img src="img/ruby_variable_1.png" /></center>

In [11]:
class SpaceShip
  def initalize(speed)
    @speed = speed
  end
    
  def accelerate(speed)
    @speed += speed
  end
end

:accelerate

Przykład wykorzystania operatora `+` zdefiniowanego dla klasy `Speed`.

# Atrybuty a dziedziczenie

In [12]:
class Speed
  attr_reader :unit
  
  def initialize(unit)
    @unit = unit
  end
end

:initialize

In [13]:
class Speed1D < Speed
  attr_reader :value
  
  def initialize(unit, value)
    super(unit)
    @value = value
    @unit = :KMH
  end
end

:initialize

`Speed1D < Speed` oznacza, że `Speed1D` dziedziczy ze `Speed`.

`super` oznacza wywołanie konstruktora z klasy nadrzędnej.

In [14]:
speed1 = Speed1D.new(:MS, 10)
speed1.unit

:KMH

<center><img src="img/ruby_class.png" /></center>

## Zunifikowany dostęp do atrybutów

In [15]:
class Speed
  attr_accessor :unit
  
  def initialize(unit)
    @unit = unit
  end
end

:initialize

`attr_accessor` definiuje zarówno getter jak i setter dla klasy `Speed`.

In [16]:
speed1 = Speed.new(:KMH)
puts speed1.unit()
speed1.unit=(:MPH)
puts speed1.unit
speed1.unit = "abc"
puts speed1.unit

KMH
MPH
abc


W Rubym nawiasy w wywołaniach metod są opcjonalne (jeśli nie powoduje to niejednoznaczności). Dlatego wywołania 
`speed.unit= value` oraz `speed.unit=(value)` a także `speed.unit` i `speed.unit()` są tożsame.

In [17]:
class Speed
  attr_accessor :unit
  VALID_UNITS = [:KMH, :MPH]

  def initialize(unit)
    @unit = unit
  end

  def unit=(new_value)
    if VALID_UNITS.include?(new_value)
      @unit = new_value
    end
  end
end

:unit=

Dzikęki temu możemy w sposób 'przezroczysty` nadpisywać metody dostępowe np. zabezpieczając przed niewłaściwymi modyfikacjami stanu naszego obiektu.

In [18]:
speed1 = Speed.new(:KMH)
puts speed1.unit
speed1.unit = :MPH
puts speed1.unit
speed1.unit = "abc"
puts speed1.unit

KMH
MPH
MPH


![Pytania? ](img/question.jpg)